In [ ]:
import logging
import yaml
import asyncio

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel import ATTCS

import SALPY_ATPtg


In [ ]:
attcs = ATTCS()

In [ ]:
attcs.log.setLevel(logging.DEBUG)

In [ ]:
await attcs.atmcs.cmd_start.start(timeout=30)

In [ ]:
for c in attcs.components():
    r = getattr(attcs, c.lower())
    try:
        state = salobj.State(r.evt_summaryState.get().summaryState)
        print(f"{c}::{state!r}")
    except AttributeError:
        print(f"{c}::UNKNOWN")

In [ ]:
attcs.ataos.evt_heartbeat.get()

In [ ]:
await attcs.ataos.evt_heartbeat.next(flush=True, timeout=10)

In [ ]:
for c in attcs.components():
    r = getattr(attcs, c.lower())
    await salobj.set_summary_state(r, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(attcs.atpneumatics, salobj.State.ENABLED)
await attcs.atpneumatics.cmd_openMasterAirSupply.start()
await attcs.atpneumatics.cmd_openInstrumentAirValve.start()
await attcs.atpneumatics.cmd_m1OpenAirValve.start()
await attcs.atpneumatics.cmd_m2OpenAirValve.start()

In [ ]:
await salobj.set_summary_state(attcs.ataos, salobj.State.ENABLED, settingsToApply='test')

In [ ]:
await salobj.set_summary_state(attcs.athexapod, salobj.State.ENABLED, timeout=300,
                               settingsToApply='Default1')

In [ ]:
await salobj.set_summary_state(attcs.atmcs, salobj.State.ENABLED, timeout=300)

In [ ]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED, timeout=300)

In [ ]:
await salobj.set_summary_state(attcs.atdome, salobj.State.ENABLED, settingsToApply='test')

In [ ]:
await salobj.set_summary_state(attcs.atdometrajectory, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [ ]:
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
# cmd_elaz = AltAz(alt=70.*u.deg, az=5.*u.deg, 
#                  obstime=curr_time_atptg.tai, 
#                  location=location)
# cmd_elaz = AltAz(alt=70.*u.deg, az=287.*u.deg, 
#                  obstime=curr_time_atptg.tai, 
#                  location=location)

cmd_elaz = AltAz(alt=70.*u.deg, az=0.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)

cmd_radec = cmd_elaz.transform_to(ICRS)

In [ ]:
await attcs.slew(ra=cmd_radec.ra.hour,
                 dec=cmd_radec.dec.deg,
                 rotPA=180.-cmd_elaz.alt.deg,
                 rot_frame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
                 rot_mode=SALPY_ATPtg.ATPtg_shared_RotMode_field)

In [ ]:
await attcs.atptg.cmd_stopTracking.start()

In [ ]:
await attcs.atdome.cmd_openShutter.start()

In [ ]:
await attcs.atdome.cmd_closeShutter.start()

In [ ]:
await salobj.set_summary_state(attcs.atdometrajectory, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY)

In [ ]:
attcs.atdome.cmd_moveAzimuth.set(azimuth=195.)

In [ ]:
await attcs.atdome.cmd_moveAzimuth.start()